In [ ]:
import os
import csv
import time
from googleapiclient.discovery import build
from datetime import datetime, timezone, timedelta

# API 정보 설정
API_SERVICE_NAME = 'youtube'
API_VERSION = 'v3'
API_KEY = ''  # 앞서 발급받은 API 키를 입력하세요.

# 라이브 스트림의 비디오 ID를 입력하세요.
VIDEO_ID = '80BCd_EKWm8' #video/ 뒤에 영어 떼기

def get_live_chat_id(youtube, video_id):
    response = youtube.videos().list(
        part='liveStreamingDetails',
        id=video_id
    ).execute()

    items = response.get('items', [])
    if not items:
        print('해당 비디오 ID에 대한 정보를 찾을 수 없습니다.')
        return None

    live_details = items[0].get('liveStreamingDetails', {})
    live_chat_id = live_details.get('activeLiveChatId')

    if not live_chat_id:
        print('라이브 채팅 ID를 찾을 수 없습니다.')
        return None

    return live_chat_id

def main():
    # YouTube API 클라이언트 생성
    youtube = build(API_SERVICE_NAME, API_VERSION, developerKey=API_KEY)

    # 라이브 채팅 ID 가져오기
    live_chat_id = get_live_chat_id(youtube, VIDEO_ID)
    if not live_chat_id:
        return

    # CSV 파일 설정
    csv_file = open('live_chat.csv', mode='w', newline='', encoding='utf-8')
    fieldnames = ['timestamp', 'author', 'author_id', 'message']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()

    next_page_token = None

    try:
        while True:
            response = youtube.liveChatMessages().list(
                liveChatId=live_chat_id,
                part='snippet,authorDetails',
                pageToken=next_page_token
            ).execute()

            for item in response.get('items', []):
                # UTC 시간의 ISO 8601 형식 문자열
                timestamp = item['snippet']['publishedAt']

                # 'Z'를 '+00:00'으로 대체하여 시간대 정보를 표준화합니다.
                timestamp = timestamp.replace('Z', '+00:00')

                # 문자열을 datetime 객체로 변환 (시간대 오프셋 포함)
                timestamp_utc = datetime.strptime(timestamp, "%Y-%m-%dT%H:%M:%S.%f%z")

                # 서울 시간대 (UTC+9) 설정
                seoul_tz = timezone(timedelta(hours=9))

                # UTC 시간을 서울 시간대로 변환
                timestamp_seoul = timestamp_utc.astimezone(seoul_tz)

                # 원하는 형식으로 시간 문자열 포맷팅
                timestamp_formatted = timestamp_seoul.strftime("%Y-%m-%d %H:%M:%S")

                author = item['authorDetails']['displayName']
                author_id = item['authorDetails']['channelId']
                message = item['snippet']['displayMessage']

                writer.writerow({
                    'timestamp': timestamp_formatted,
                    'author': author,
                    'author_id': author_id,
                    'message': message
                })
                print(f"{timestamp_formatted} [{author}] ({author_id}) - {message}")

            next_page_token = response.get('nextPageToken')
            polling_interval = response['pollingIntervalMillis'] / 1000

            time.sleep(polling_interval)
    except KeyboardInterrupt:
        print("프로그램이 사용자에 의해 중단되었습니다.")
    except Exception as e:
        print(f"오류가 발생했습니다: {e}")
    finally:
        csv_file.close()

if __name__ == '__main__':
    main()
